<a href="https://colab.research.google.com/github/Arthur-Povill/LeiloesTDSat/blob/main/Case_de_Predi%C3%A7%C3%A3o_de_Fraudes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case de Predição de Fraudes em Transações de Cartão de Crédito 💳

## Contexto:

Você é um(a) cientista de dados em uma instituição financeira renomada. A empresa está enfrentando um aumento significativo nas transações fraudulentas de cartões de crédito, o que está prejudicando a confiança dos clientes e causando prejuízos financeiros. Sua tarefa é desenvolver um modelo preditivo que possa identificar transações fraudulentas com alta precisão, minimizando assim o impacto financeiro e protegendo a reputação da instituição.

## Base de Dados:

Você tem acesso à base de dados "**fraud_dataset.csv**", que contém informações detalhadas sobre as transações de cartões de crédito.

## Objetivo:

Desenvolver um modelo preditivo utilizando o algoritmo de Regressão Logística com regularização L2 para prever se uma transação de cartão de crédito é fraudulenta ou não.

**Realize os seguintes passos na sua entrega:**

- Pré-processamento de Dados:
  - Verifique se a base de dados contém informações nulas.
  - Considere a padronização de variáveis numéricas.
- Divisão da Base de Dados:
  - Separe a base de dados utilizando a técnica hold out, mantendo uma proporção adequada para garantir a representatividade dos dados.
  - Caso a base de dados tenha algum tipo de desequilíbrio, aplique uma técnica de oversampling para equilibrar as classes.
- Modelagem:
  - Implemente o algoritmo de Regressão Logística com regularização L2.
  - Utilize a técnica de validação cruzada k-fold (com 5 folds) para avaliar o desempenho do modelo.
- Avaliação do Modelo:
  - Avalie o modelo utilizando métricas como precisão, recall, F1-score e área sob a curva ROC (AUC-ROC).
- Interpretação e Comunicação de Resultados:
  - Comunique os resultados de forma clara e acessível para as partes interessadas, destacando a eficácia do modelo na identificação de transações fraudulentas.


In [9]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Pré-processamento e modelagem
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Balanceamento de classes
from imblearn.over_sampling import SMOTE

Subir o banco de dados e verificar se há dados nulos

In [10]:
#leitura do dataset
df = pd.read_csv('/content/sample_data/fraud_dataset.csv')

#verifição de dados ausentes
df.isnull().sum()


,0
distance_from_home,0
distance_from_last_transaction,0
ratio_to_median_purchase_price,0
repeat_retailer,0
used_chip,0
used_pin_number,0
online_order,0
fraud,0


Verificação se há desequilíbrio de Classe - Fraud
1.0 == Há fraude
0.0 == Não há fraude


In [11]:
# Verificando o balanceamento da variável alvo
round(df["fraud"].value_counts(normalize = True)*100,2)


,proportion
fraud,
0.0,91.26
1.0,8.74


Separação da base em train vs Test com Técnica de Holdout(80% Treino x 20% teste)

In [12]:
# Separando as variáveis independentes (X) e dependente (y)
X = df.drop('fraud', axis = 1)
y = df['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y) # garante a mesma proporção de classes em ambos os conjuntos


Aplicação da Padronização

In [13]:
# Padronizando os dados para que todas as variáveis tenham média 0 e desvio padrão 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Aplicando o Oversampling

In [14]:
# Aplicando SMOTE apenas no conjunto de treino
smote = SMOTE(random_state = 42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Verificando nova distribuição após o balanceamento
print(pd.Series(y_train).value_counts(normalize = True)* 100,2)

fraud
0.0    50.0
1.0    50.0
Name: proportion, dtype: float64 2


Regressão Logística(Regularização L2) e Validação Cruzada

In [15]:
# Instanciando o modelo
model = LogisticRegression(penalty = "l2")

# Validação cruzada estratificada
skfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

# Avaliação usando AUC como métrica
score = cross_val_score(model, X_train, y_train, cv = skfold, scoring= "roc_auc")

# Treinando o modelo com os dados balanceados
model.fit(X_train, y_train)

# Exibindo a média dos scores de validação
np.mean(score)

np.float64(0.9793964360941153)

Chamada do Predict

In [19]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # probabilidades da classe positiva

Avaliação do Modelo

In [20]:

# Métricas de desempenho
auc_score = roc_auc_score(y_test, y_pred_proba)
print(classification_report(y_test, y_pred))
print(f"AUC Score: {auc_score:.4f}")
print(f"Mean Cross-Validation AUC: {np.mean(score):.4f}")


              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    182519
         1.0       0.58      0.95      0.72     17481

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000

AUC Score: 0.9795
Mean Cross-Validation AUC: 0.9794
